# Flood ConvLSTM training

In [1]:
# Setup with GPU
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%load_ext autoreload
%autoreload 2
import tensorflow as tf
tf.config.list_physical_devices('GPU')

env: TF_FORCE_GPU_ALLOW_GROWTH=true


2024-07-15 17:46:54.310239: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-15 17:46:54.310283: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-15 17:46:54.311732: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-15 17:46:54.319891: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-15 17:46:56.556837: I external/local_xla/xla/

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
# Load the dataset form firestore
from usl_models.flood_ml.trainingdataset import IncrementalTrainDataGenerator

sim_names = ["Manhattan-config_v1%2FRainfall_Data_1.txt", "Manhattan-config_v1%2FRainfall_Data_2.txt"]
batch_size = 1 # On A100, use batch_size = 4

generator = IncrementalTrainDataGenerator()
dataset = generator.load_dataset_windowed(sim_names, batch_size=batch_size, max_chunks=8)


2024-07-15 17:46:56.757216: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-15 17:46:56.760599: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-15 17:46:56.763692: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [3]:
# Initialize the model
from usl_models.flood_ml.model import FloodModel
from usl_models.flood_ml.model import Input
from usl_models.flood_ml.model_params import FloodModelParams
import keras
model = FloodModel(
    model_params=FloodModelParams(batch_size=batch_size))

In [4]:
# Fit the model to the dataset
callbacks = [keras.callbacks.TensorBoard(log_dir='./logs',
                                         histogram_freq=1,
                                         profile_batch='1,100')]
model.fit(dataset, epochs=1, callbacks=callbacks)

2024-07-15 17:46:57.706604: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-15 17:46:57.706639: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-15 17:46:57.706682: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1883] Profiler found 1 GPUs
2024-07-15 17:46:57.728617: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-07-15 17:46:57.728861: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:2017] CUPTI activity buffer flushed
2024-07-15 17:46:57.773766: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-15 17:46:57.773797: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-15 17:47:01.258020: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match siz

Generating feature and label tensors...
Fetching rainfall configuration...
Configuration:  config_v1%2FRainfall_Data_1.txt
Rainfall duration:  19
The output label tensor will initially be of shape:  [1000, 1000, 19]
Creating feature chunks...
Creating label chunks...
Length of common indices: 66
Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_5_3.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/5_3.npy
Finished creating feature and label tensors...
Feature tensor shape: (1000, 1000, 8)
Label tensor shape: (19, 1000, 1000)


Generating temporal tensors...
Creating temporal chunks...
Configuration:  config_v1%2FRainfall_Data_1.txt
Fetching document: config_v1%2FRainfall_Data_1.txt
Loading temporal tensor from gs://test-climateiq-study-area-feature-chunks/rainfall/config_v1/Rainfall_Data_1.npy...
Temporal tensor shape:  (864, 6)


2024-07-15 17:47:06.537052: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-07-15 17:47:10.439444: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f643c010db0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-15 17:47:10.439487: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-07-15 17:47:10.446496: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1721065630.570962   11184 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-07-15 17:47:11.118153: W external/local_tsl/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.50GiB with freed_by_count=0. The caller indicates that this is no

     18/Unknown - 26s 557ms/step - loss: 5.5360e-06 - mean_absolute_error: 4.7806e-04 - root_mean_squared_error: 0.0024Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_3_0.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/3_0.npy
     19/Unknown - 27s 557ms/step - loss: 5.2446e-06 - mean_absolute_error: 4.5290e-04 - root_mean_squared_error: 0.0023Finished creating feature and label tensors...
Feature tensor shape: (1000, 1000, 8)
Label tensor shape: (19, 1000, 1000)


     36/Unknown - 37s 568ms/step - loss: 2.7680e-06 - mean_absolute_error: 2.3903e-04 - root_mean_squared_error: 0.0017Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_0_5.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/0_5.npy
     38/Unknown - 38s 568ms/step - loss: 2.6223e-06 - mean_absolute_error: 2.2645e-04 - root_mean_squared_error: 0.0016Finished c

2024-07-15 17:48:12.236750: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-07-15 17:48:12.250989: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:2017] CUPTI activity buffer flushed
2024-07-15 17:48:13.746820: I external/local_xla/xla/backends/profiler/gpu/cupti_collector.cc:541]  GpuTracer has collected 275744 callback api events and 273748 activity events. 
2024-07-15 17:48:16.236806: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-07-15 17:48:16.238484: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: ./logs/plugins/profile/2024_07_15_17_48_16/dl-m123-a100.xplane.pb


    110/Unknown - 86s 632ms/step - loss: 0.0061 - mean_absolute_error: 0.0098 - root_mean_squared_error: 0.0778Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_5_6.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/5_6.npy
    112/Unknown - 87s 632ms/step - loss: 0.0063 - mean_absolute_error: 0.0101 - root_mean_squared_error: 0.0793Finished creating feature and label tensors...
Feature tensor shape: (1000, 1000, 8)
Label tensor shape: (19, 1000, 1000)


    129/Unknown - 97s 630ms/step - loss: 0.0057 - mean_absolute_error: 0.0097 - root_mean_squared_error: 0.0758Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_3_8.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/3_8.npy
    131/Unknown - 98s 629ms/step - loss: 0.0057 - mean_absolute_error: 0.0095 - root_mean_squared_error: 0.0752Finished creating feature and label tensor

In [ ]:
dataset = generator.load_dataset_windowed(sim_names, batch_size=1, max_chunks=8)
input, labels = next(iter(dataset))
prediction = model.call(input)
prediction.shape, tf.math.reduce_max(prediction)

Generating feature and label tensors...
Fetching rainfall configuration...
Configuration:  config_v1%2FRainfall_Data_1.txt
Rainfall duration:  19
The output label tensor will initially be of shape:  [1000, 1000, 19]
Creating feature chunks...
Creating label chunks...
Length of common indices: 66
Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_2_4.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/2_4.npy
Finished creating feature and label tensors...
Feature tensor shape: (1000, 1000, 8)
Label tensor shape: (19, 1000, 1000)


Generating temporal tensors...
Creating temporal chunks...
Configuration:  config_v1%2FRainfall_Data_1.txt
Fetching document: config_v1%2FRainfall_Data_1.txt
Loading temporal tensor from gs://test-climateiq-study-area-feature-chunks/rainfall/config_v1/Rainfall_Data_1.npy...
Temporal tensor shape:  (864, 6)


(TensorShape([1, 1000, 1000, 1]),
 <tf.Tensor: shape=(), dtype=float32, numpy=0.08844803>)

In [ ]:
model.save_model('data/flood_model_tf213_20240711-190530')

In [ ]:
# Test call_n
dataset = generator.load_dataset(sim_names, batch_size=1, max_chunks=1)
input, labels = next(iter(dataset))

(B, n, H, W) = labels.shape
predictions = model.call_n(input, n=n)
predictions.shape, tf.math.reduce_max(predictions)

Generating feature and label tensors...
Fetching rainfall configuration...
Configuration:  config_v1%2FRainfall_Data_1.txt
Rainfall duration:  19
The output label tensor will initially be of shape:  [1000, 1000, 19]
Creating feature chunks...
Creating label chunks...
Length of common indices: 66
Feature file: gs://test-climateiq-study-area-feature-chunks/Manhattan/scaled_chunk_2_3.npy
Label file: gs://test-climateiq-study-area-label-chunks/Manhattan/config_v1/Rainfall_Data_1.txt/2_3.npy


2024-07-12 01:53:53.465962: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76000000 exceeds 10% of free system memory.


Finished creating feature and label tensors...
Feature tensor shape: (1000, 1000, 8)
Label tensor shape: (19, 1000, 1000)


Generating temporal tensors...
Creating temporal chunks...
Configuration:  config_v1%2FRainfall_Data_1.txt
Fetching document: config_v1%2FRainfall_Data_1.txt
Loading temporal tensor from gs://test-climateiq-study-area-feature-chunks/rainfall/config_v1/Rainfall_Data_1.npy...
Temporal tensor shape:  (864, 6)


2024-07-12 01:53:54.273372: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76000000 exceeds 10% of free system memory.
2024-07-12 01:53:54.307598: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 76000000 exceeds 10% of free system memory.
2024-07-12 01:53:54.498488: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


(TensorShape([1, 19, 1000, 1000]),
 <tf.Tensor: shape=(), dtype=float32, numpy=0.011001954>)